In [34]:
import pandas as pd
import numpy as np

df = pd.read_csv(r'2023_20240309_40664.csv', encoding='euc-kr')
# df.columns.values
df

C:\Users\SeanJ\AppData\Local\Temp\ipykernel_19776\1436740347.py:4: DtypeWarning: Columns (62,63,71,72,89,90) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'2023_20240309_40664.csv', encoding='euc-kr')


In [50]:
# 비어있는 'df_member' 데이터프레임 생성
df_member = pd.DataFrame()

# 가구원2 부터 가구원9 이름 바꾸고 합치기
for mem in range(2,10):
    # e.g. '가구원2_' 가 들어가는 열만 찾기
    df_temp = df.filter(like=f'가구원{mem}_')
    # 이름 바꾸기
    df_temp.columns = [col.replace(f'가구원{mem}_','') for col in df_temp.columns]
    # 'df_member' 데이터프레임에 합치기
    df_member = pd.concat([df_member,df_temp])

df_member

# 필터링 - 배우자 (가구원_가구주관계코드가 2인 경우만)
df_member = df_member.loc[df_member['가구주관계코드']==2]

# 합치기 - 가구정보 및 배우자 소득
df_member = df_member.merge(df[['조사연월','가구일련번호','소득_경상소득_근로소득_배우자근로소득']], how='left', left_index=True, right_index=True)

# 이름바꾸기
df_member.rename(columns={'소득_경상소득_근로소득_배우자근로소득':'경상근로소득'}, inplace=True)
df_member.columns = [col.replace('가구주','') for col in df_member.columns]



df_member

,관계코드,성별코드,연령,학력코드,수학구분코드,취업여부,10차산업분류코드,7차직업분류코드,종사상지위코드,조사연월,가구일련번호,경상근로소득
0,2.0,2.0,43.0,4.0,1.0,1.0,G,5,5.0,202314,1,0
1,2.0,2.0,62.0,3.0,4.0,2.0,Z,Z,NaN,202314,2,0
2,2.0,1.0,56.0,4.0,1.0,1.0,R,4,5.0,202314,3,48500
6,2.0,2.0,35.0,4.0,1.0,1.0,G,5,5.0,202314,7,0
8,2.0,2.0,56.0,4.0,1.0,1.0,S,3,4.0,202314,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...
27655,2.0,2.0,39.0,6.0,1.0,1.0,P,3,2.0,202344,7118,135000
27660,2.0,1.0,58.0,3.0,1.0,1.0,H,8,5.0,202344,7123,0
27661,2.0,2.0,77.0,2.0,1.0,1.0,Q,9,2.0,202344,7124,270000
27665,2.0,2.0,70.0,4.0,1.0,1.0,A,6,6.0,202344,7128,0


In [49]:
# 가구주 정보 필터링
df_household = df.filter(like='가구주_').merge(df[['조사연월','가구일련번호','소득_경상소득_근로소득_가구주근로소득']], how='left', left_index=True, right_index=True)

# 이름 바꾸기
df_household.rename(columns={'소득_경상소득_근로소득_가구주근로소득':'경상근로소득'}, inplace=True)

# 가구주 이름 제거
df_household.columns = [col.replace('가구주_','') for col in df_household.columns]

df_household

,관계코드,성별코드,연령,학력코드,수학구분코드,취업여부,10차산업분류코드,7차직업분류코드,종사상지위코드,조사연월,가구일련번호,경상근로소득
0,1,1,43,5,4,1,F,3,1.0,202314,1,5170000
1,1,1,66,3,1,1,N,9,1.0,202314,2,2176800
2,1,2,51,4,1,1,L,2,5.0,202314,3,0
3,1,2,50,4,1,1,C,7,3.0,202314,4,2078090
4,1,1,30,6,4,2,Z,Z,NaN,202314,5,700514
...,...,...,...,...,...,...,...,...,...,...,...,...
27663,1,2,80,2,1,2,Z,Z,NaN,202344,7126,0
27664,1,2,56,4,1,1,A,6,5.0,202344,7127,2061506
27665,1,1,73,6,1,1,A,6,5.0,202344,7128,0
27666,1,1,47,4,1,1,F,8,5.0,202344,7129,0


In [63]:
df_export = pd.concat([df_household,df_member])
df_export.rename(
                columns={ 
                    '관계코드'        : 'household',
                    '조사연월'        :'date', 
                    '가구일련번호'    :'id', 
                    '성별코드'         :'female',
                    '연령'             :'age',
                    '학력코드'         :'education',
                    '수학구분코드'     :'edu_complete',
                    '경상근로소득'  :'income',
                          }, inplace=True)
df_export = df_export[['date', 'id','household', 'female','age','education','edu_complete','income']]

# date
yq = df_export['date']#.astype(str)
df_export['date'] = list(map(lambda x : '{y}-{m}-1'.format(y=str(x)[:4], m=((int(str(x)[4:5]))*3)), yq))
df_export['date'] = pd.to_datetime(df_export['date'])

# household
df_export['household'] = list(map(lambda x : x==1 , df_export['household']))

# female
df_export['female'] = list(map(lambda x : x==2 , df_export['female']))

# education
#  
def edu_year (edu_level, edu_comp):
    ref = pd.DataFrame({'edu_year':[0,6,9,12,14,16,18,21]})
    if edu_level == 1 :
        edu_year = 0
    else :
        if edu_comp == 1 :
            edu_year = ref.loc[edu_level-1,'edu_year']
        else :
            edu_year = ref.loc[edu_level-2,'edu_year']
    return edu_year
# 
df_export['edu_year'] = list(map(lambda x : edu_year(x[0],x[1]), zip(df_export['education'],df_export['edu_complete'])))
df_export.drop(columns=['education','edu_complete'],inplace=True)

# income float
df_export['income'] = df_export['income'].astype('float64')
# drop 0 income
df_export = df_export.loc[df_export['income']>0]
# add log income
df_export['ln_income'] = np.log(df_export['income'])

# reset_index
df_export.reset_index(drop=True,inplace=True)

df_export

,date,id,household,female,age,income,edu_year,ln_income
0,2023-03-01,1,True,False,43.0,5170000.0,12,15.458383
1,2023-03-01,2,True,False,66.0,2176800.0,9,14.593366
2,2023-03-01,4,True,True,50.0,2078090.0,12,14.546960
3,2023-03-01,5,True,False,30.0,700514.0,14,13.459570
4,2023-03-01,6,True,True,65.0,895976.0,12,13.705669
...,...,...,...,...,...,...,...,...
22052,2023-12-01,7109,False,True,39.0,2500000.0,16,14.731801
22053,2023-12-01,7114,False,True,51.0,2292890.0,12,14.645324
22054,2023-12-01,7118,False,True,39.0,135000.0,16,11.813030
22055,2023-12-01,7124,False,True,77.0,270000.0,6,12.506177


In [64]:
df_export.to_csv(r'data_income_kor.csv')
df_export.to_excel(r'data_income_kor.xlsx')